# Draw Countries Polygons From a File GeoDatabase

In [ ]:
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
from arcgis.geometry import Polygon

Read a GDB table as Spark DataFrame and create `Countries` temp table.

In [ ]:
spark.read \
    .format('com.esri.gdb') \
    .options(path='World.gdb', name='Countries') \
    .load() \
    .registerTempTable('Countries')

Read the `name` and `shape` and convert it to a Pandas DataFrame.

In [ ]:
pdf = sql("select ADMIN_NAME,Shape.parts as parts, Shape.coords as coords from Countries").toPandas()

Function to convert `parts` array\[Int\] and `coords` array\[Double\] to a `Polygon` instance.

In [ ]:
def to_geom(parts,coords):
    rings = []
    i = 0
    j = 2
    for num_points in parts:
        ring = []
        n = 0
        while n < num_points:
            ring.append(coords[i:j])
            i += 2
            j += 2
            n += 1
        rings.append(ring)
    return Polygon({"spatialReference" : {"wkid" : 4326}, "rings": rings})

Convert `parts` and `coords` to a `Polygon` for each row and create the required `SHAPE` column.

In [ ]:
pdf['SHAPE'] = pdf.apply(lambda row: to_geom(row.parts,row.coords), axis=1)

Drop the `parts` and `coords` columns, and create a `SpatialDataFrame`.

In [ ]:
sdf = SpatialDataFrame(pdf.drop(['parts','coords'], axis=1))

Create a map.

In [ ]:
gis = GIS()
m = gis.map()
m.basemap = 'dark-gray'
m

Plot the Spatial DF on the map

In [ ]:
sdf.plot(kind='map', map_widget=m, alpha=0.4)